In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltz
from apyori import apriori

# Указание правильного разделителя и обработка некорректных строк
data = pd.read_csv('Dataset9_2.csv', sep=',', on_bad_lines='skip')

data.head()


,Id,Price,Year,Mileage,City,State,Vin,Make,Model
0,1,16472,2015,18681,Jefferson City,MO,KL4CJBSBXFB267643,Buick,EncoreConvenience
1,2,15749,2015,27592,Highland,IN,KL4CJASB5FB245057,Buick,EncoreFWD
2,3,16998,2015,13650,Boone,NC,KL4CJCSB0FB264921,Buick,EncoreLeather
3,4,15777,2015,25195,New Orleans,LA,KL4CJASB4FB217542,Buick,EncoreFWD
4,5,16784,2015,22800,Las Vegas,NV,KL4CJBSB3FB166881,Buick,EncoreConvenience


In [43]:
data = data.astype(str)
# Создаем новые столбцы
data['Make + Model'] = data['Make'] + ' ' + data['Model']
data['City + State'] = data['City'] + ' ' + data['State']

# Удаляем ненужные столбцы
data.drop(['Make', 'Model', 'Vin', 'City', 'State', 'Id'], axis=1, inplace=True)
# Преобразуем все значения в строки

#  Преобразуем DataFrame в список списков (transactions)
transactions = data.values.tolist()
# Выводим первые несколько строк для проверки
data.head()


#

,Price,Year,Mileage,Make + Model,City + State
0,16472,2015,18681,Buick EncoreConvenience,Jefferson City MO
1,15749,2015,27592,Buick EncoreFWD,Highland IN
2,16998,2015,13650,Buick EncoreLeather,Boone NC
3,15777,2015,25195,Buick EncoreFWD,New Orleans LA
4,16784,2015,22800,Buick EncoreConvenience,Las Vegas NV




В данном коде реализован алгоритм ассоциативного анализа а-priori для выявления связей между продуктами. Алгоритм выполняет следующие действия:

Поиск элементов, поддерживающих связь с определенным продуктом. Элементы с меньшей поддержкой исключаются из рассмотрения.

Создание связей между элементами, если они поддерживают связь с определенным продуктом и их достоверность больше определенного значения.

Упорядочение связей по уровню поддержки, достоверности и показателю "подъем".

Возвращение списка найденных связей.

Далее в коде реализована функция inspect для проверки и возврата информации о найденных связях. В итоге все найденные связи преобразуются в DataFrame для удобного отображения.

Последние строки кода позволяют отобразить 10 самых интересных связей (по показателю "подъем") между продуктами.

In [42]:
rules = apriori(transactions=transactions, min_support = 0.001, min_confidence=0.001, min_lift=1.1, min_length =2, max_length=2)
result = list(rules)

print(result[0])

RelationRecord(items=frozenset({'2011', 'Chevrolet Silverado'}), support=0.0017290853031860225, ordered_statistics=[OrderedStatistic(items_base=frozenset({'2011'}), items_add=frozenset({'Chevrolet Silverado'}), confidence=0.036699008795197545, lift=1.233901958510505), OrderedStatistic(items_base=frozenset({'Chevrolet Silverado'}), items_add=frozenset({'2011'}), confidence=0.0581356775584674, lift=1.233901958510505)])


In [44]:
def inspect(results):
    lhs = [tuple(result[2][0][0])[0] for result in results]
    rhs = [tuple(result[2][0][1])[0] for result in results]
    supports = [result[1] for result in results]
    confidence = [result[2][0][2] for result in results]
    lifts = [result[2][0][3] for result in results]
    return list (zip(lhs,rhs,supports,confidence,lifts))

resultsinDataFrame = pd.DataFrame(inspect(result), columns=["Left hand side", "Right hand side", "Support", "Confidence", "lifts"])

resultsinDataFrame = resultsinDataFrame.nlargest(n=10,columns="lifts")

resultsinDataFrame

#lift - это отношение зависимости, Показывает насколько items зависят друг от друга  
#support - показатель частности 
#confidence - показатель уверенности, как часто наше правило срабатывает для всего датасета


,Left hand side,Right hand side,Support,Confidence,lifts
5,2013,Nissan AltimaSedan,0.001399,0.016331,11.669913
2,2013,Ford F-150SuperCrew,0.001972,0.023018,11.660192
118,2017,Ram 1500SLT,0.001306,0.011164,8.529265
117,2017,Mercedes-Benz C-ClassC,0.001482,0.012676,8.378805
102,2017,Chevrolet MalibuLT,0.005041,0.043104,7.271102
1,2011,Hyundai Sonata4dr,0.001034,0.021953,6.763032
101,2017,Chevrolet CruzeSedan,0.002917,0.024944,6.737299
67,2016,Chevrolet Cruze,0.002089,0.013437,6.431578
68,2016,Chevrolet Malibu,0.001386,0.008916,6.047764
115,2017,Kia SedonaLX,0.001329,0.011361,5.798578


In [71]:
import plotly.express as px

data_10_best = resultsinDataFrame[:10]
fig = px.scatter(data_10_best, x = 'Support', y = 'Confidence', color='lifts')
fig1 = px.scatter(data_10_best, x = 'Left hand side', y = 'Right hand side', color='lifts', size='Support')
fig.show()
fig1.show()
